In [5]:
import pandas as pd
import random

In [6]:
# Veut dire que c’est un passage par adresse des 3 variables IX, IY et IZ.
# 3 variables entières qu’il faut initialiser avant le 1er appel de la fonction.
# Veut dire que la fonction est typée, elle retourne un nombre réel.
def alea(IX, IY, IZ):
    # Déclaration d’une variable inter de type réel
    inter = 0.0
    # A chaque exécution de la fonction, les valeurs des 3 variables sont recalculées, d’où un passage par adresse.
    # % : division entière
    # // : reste de la division entière
    IX = 171 * (IX % 177) - 2 * (IX // 177)
    IY = 172 * (IY % 176) - 35 * (IY // 176)
    IZ = 170 * (IZ % 178) - 63 * (IZ // 178)
    # S’il y a un dépassement de capacité sur une variable, on ajoute un nombre très grand, pour la machine 16 bits,
    # pour ramener à une valeur positive. Ce qui marche sur une machine 16 bits marche sur une 32 ou 64 bits.
    if IX < 0:
        IX = IX + 30269
    if IY < 0:
        IY = IY + 30307
    if IZ < 0:
        IZ = IZ + 30323
    inter = ((IX / 30269) + (IY / 30307) + (IZ / 30323))
    return [round(inter - int(inter), 4),IX, IY, IZ]

In [7]:
IX = 100
IY = 100
IZ = 100

In [8]:
salaire = [(40000, 30000), (30000, 20000), (20000, 15000), (15000, 10000), (10000, 7500), (7500, 5000), (5000, 3000)]
frequenceSalaire = [5, 10, 10, 15, 20, 20, 20]
age = [(61, 53), (52, 41), (40, 31), (30, 21)]
frequenceAge = [30, 30, 20, 20]
ageEmbauche = [(21, 22), (23, 24), (25, 28), (29, 32), (33, 35), (36, 40)]
frequenceAgeEmbauche = [5, 20, 25, 25, 20, 5]
salaire_embauche=[(32000,24000),(24000,16000),(16000,12000),(12000,8000),(8000,6000),(6000,4000),(4000,3000)]
frequenceSalaireEmbauche=[5,5,10,20,20,20,20]
years = [24, 25, 26, 27, 28, 29, 30, 31, 32, 33]

In [9]:
ensemble_data_frames = {}
for j in range(1, 41):
    data_frames = {}
    for i in range(23, 34) :
        data_frames['Employe_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire", "cotisation"])
        data_frames['Retraite_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire"])
        data_frames['newRecru_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire", "cotisation"])
    ensemble_data_frames['Simulation_'+str(j)] = data_frames

In [10]:
data_indices = {}
for j in range(1, 41):
    data_indices['Simulation_'+str(j)] = pd.DataFrame(columns=["year", "TotEmp", "TotRet", "TotCotis", "TotPens", "Reserve", "NouvRet", "NouvRec"])

In [11]:
def cumuleFrequenceIntervale(frequences):
    cumules = []
    index = 0
    for i in frequences:
        cumules.append([index, i+index])
        index += i
    return cumules

In [12]:
def indexFromTable(cumules, aleaNumber):
    i = 0
    for intervale in cumules :
        if intervale[1] > aleaNumber*100 :
            return i
        i+=1
    return i

In [13]:
def cotisation(salaire):
    if salaire < 5000:
        return salaire*0.05
    elif salaire < 7000 :
        return salaire*0.06
    elif salaire < 100000 :
        return salaire*0.07
    else:
        return salaire*0.1

In [14]:
def createEmployeRetraite(simulation, year, IX, IY, IZ):
    dfEmploye = ensemble_data_frames["Simulation_"+str(simulation)]["Employe_20"+str(year)]
    dfRetraite = ensemble_data_frames["Simulation_"+str(simulation)]["Retraite_20"+str(year)]
    for index in range(11000):
        if index >=1:
            IX = alea(IX, IY, IZ)[1]
            IY = alea(IX, IY, IZ)[2]
            IZ = alea(IX, IY, IZ)[3]
        salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaire), alea(IX, IY, IZ)[0])
        ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAge), alea(IX, IY, IZ)[0])
        i = index+1
        s = random.randrange(salaire[salaireIndex][1], salaire[salaireIndex][0], 100)
        if index <10000:
            a = random.randrange(age[ageIndex][1], age[ageIndex][0], 1)

            new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s], "cotisation" : [cotisation(s)]})
            dfEmploye = pd.concat([dfEmploye, new_row], ignore_index=True)
        else:
            a = random.randint(65, 85)
            new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
            dfRetraite = pd.concat([dfRetraite, new_row], ignore_index=True)
    ensemble_data_frames["Simulation_"+str(simulation)]["Employe_20"+str(year)] = dfEmploye
    ensemble_data_frames["Simulation_"+str(simulation)]["Retraite_20"+str(year)] = dfRetraite

In [15]:
def pensionnaireFive(dfRetrait) :
    pensionnaire65 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    for index, row in dfRetrait.iterrows():
        a = row["age"]
        s = row["salaire"]
        p = ((25*2)/100)*row["salaire"]
        new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
        pensionnaire65 = pd.concat([pensionnaire65, new_row], ignore_index=True)
    return pensionnaire65

In [16]:
def pensionnaireTwo(dfEmploye, dfRetraite):
    pensionnaire62 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    pensionnaire65 = pensionnaireFive(dfRetraite)
    pensionnaire62 = pd.concat([pensionnaire62, pensionnaire65], ignore_index=True)
    for index, row in dfEmploye.iterrows():
        if row["age"]>=62:
            a = row["age"]
            s = row["salaire"]
            p = ((25*2)/100)*row["salaire"]
            new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
            pensionnaire62 = pd.concat([pensionnaire62, new_row], ignore_index=True)
    return pensionnaire62

In [17]:
def newRecrutement(IX, IY, IZ):
    Nouveux_Recrus = pd.DataFrame(columns=["id", "age", "salaire"])
    a=alea(IX, IY, IZ)[0]
    n = ((400 - 250) * a) + 250
    for index in range(int(n)):
        if index >= 0 :
            IX = alea(IX, IY, IZ)[1]
            IY = alea(IX, IY, IZ)[2]
            IZ = alea(IX, IY, IZ)[3]
        salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaireEmbauche), alea(IX, IY, IZ)[0])
        ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAgeEmbauche), alea(IX, IY, IZ)[0])
        i = index+1
        s = random.randrange(salaire_embauche[salaireIndex][1], salaire_embauche[salaireIndex][0], 100)
        a = random.randrange(ageEmbauche[ageIndex][0], ageEmbauche[ageIndex][1], 1)
        new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s], "cotisation" : [cotisation(s)]})
        Nouveux_Recrus = pd.concat([Nouveux_Recrus, new_row], ignore_index=True)
    return Nouveux_Recrus

In [18]:
def calculEmployeRetraite(simulation, i, IX, IY, IZ):
    EmployeBeta = ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i-1)]
    RetraiteBeta = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i-1)]
    EmployeAlpha = ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i)]
    RetraiteAlpha = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i)]
    for index, row in EmployeBeta.iterrows():
        if row["age"] < 64:
            if i == 25 or i == 30:
                s = row["salaire"]*1.05
            else :
                s = row["salaire"]
            a = row["age"]+1
            new_row = pd.DataFrame({"id" : row["id"], "age": [a], "salaire": [s], "cotisation" : [cotisation(s)]})
            EmployeAlpha = pd.concat([EmployeAlpha, new_row], ignore_index=True)
        else :
            if i == 25 or i == 30:
                s = row["salaire"]*1.05
            else :
                s = row["salaire"]
            a = row["age"]+1
            new_row = pd.DataFrame({"id" : row["id"], "age": [a], "salaire": [s]})
            RetraiteAlpha = pd.concat([RetraiteAlpha, new_row], ignore_index=True)
    for index, row in RetraiteBeta.iterrows():
        new_row = pd.DataFrame({"id" : [row["id"]], "age": [row["age"]+1], "salaire": [row["salaire"]]})
        RetraiteAlpha = pd.concat([RetraiteAlpha, new_row], ignore_index=True)
    newRecru = newRecrutement(IX, IY, IZ)
    for index, row in newRecru.iterrows():
        new_row = pd.DataFrame({"id" : [row["id"]], "age": [row["age"]], "salaire": [row["salaire"]],"cotisation" : [row["cotisation"]]})
        EmployeAlpha = pd.concat([EmployeAlpha, new_row], ignore_index=True)
    ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i)] = EmployeAlpha
    ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i)] = RetraiteAlpha
    ensemble_data_frames["Simulation_"+str(simulation)]['newRecru_20'+str(i)] = newRecru

In [19]:
def calculIndice(simulation, ageRetrait):
    dfIndice = data_indices["Simulation_"+str(simulation)]
    dfEmploye =  ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(23)]
    dfRetraite = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(23)]
    TotEmp = dfEmploye.shape[0]
    if ageRetrait == 62:
        TotCotis = dfEmploye[dfEmploye["age"]<=61]["cotisation"].sum()*12
        pensionnaire = pensionnaireTwo(dfEmploye, dfRetraite)
    else:
         TotCotis = dfEmploye["cotisation"].sum()*12
         pensionnaire = pensionnaireFive(dfRetraite)
    TotRet = pensionnaire.shape[0]
    TotPens = pensionnaire["pension"].sum()*12
    Reserve = 200000000 + TotCotis - TotPens
    NouvRet = 0
    NouvRec = 0
    new_row = pd.DataFrame({"year" : [2023], "TotEmp" : [TotEmp], "TotRet" : [TotRet], "TotCotis" : [TotCotis], "TotPens" : [TotPens], "Reserve" : [Reserve], "NouvRet" : [NouvRet], "NouvRec" : [NouvRec]})
    dfIndice = pd.concat([dfIndice, new_row], ignore_index=True)
    for year in years :
        dfEmploye =  ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(year)]
        dfRetraite = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)]
        dfRecrutement = ensemble_data_frames["Simulation_"+str(simulation)]['newRecru_20'+str(year)]
        TotEmp = dfEmploye.shape[0]
        if ageRetrait == 62:
            TotCotis = dfEmploye[dfEmploye["age"]<=61]["cotisation"].sum()*12
            pensionnaire = pensionnaireTwo(dfEmploye, dfRetraite)
        else:
            pensionnaire = pensionnaireFive(dfRetraite)
            TotCotis = dfEmploye["cotisation"].sum()*12
        TotPens = pensionnaire["pension"].sum()*12
        TotRet = pensionnaire.shape[0]
        Reserve = dfIndice.iloc[year-24]['Reserve'] + TotCotis -TotPens
        NouvRet = TotRet - dfIndice.iloc[year-24]['TotRet']
        NouvRec =  dfRecrutement.shape[0]
        new_row = pd.DataFrame({"year" : [2000+year], "TotEmp" : [TotEmp], "TotRet" : [TotRet], "TotCotis" : [TotCotis], "TotPens" : [TotPens], "Reserve" : [Reserve], "NouvRet" : [NouvRet], "NouvRec" : [NouvRec]})
        dfIndice = pd.concat([dfIndice, new_row], ignore_index=True)
    return dfIndice

In [20]:
createEmployeRetraite(1, 23, IX, IY, IZ)
for year in years:
    calculEmployeRetraite(1, year, IX, IY, IZ)

In [21]:
simulation = 1
year = 33
ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(year)]

,id,age,salaire,cotisation
0,1,47,7607.25,532.5075
1,2,61,8268.75,578.8125
2,4,36,3748.50,187.4250
3,5,38,4299.75,214.9875
4,6,39,4740.75,237.0375
...,...,...,...,...
11260,349,29,6600.00,396.0000
11261,350,31,4400.00,220.0000
11262,351,34,4700.00,235.0000
11263,352,23,19800.00,1386.0000


In [22]:
ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)]

,id,age,salaire
0,86,65,35390.25
1,107,65,39910.50
2,110,65,17970.75
3,160,65,20286.00
4,179,65,16427.25
...,...,...,...
3260,10996,91,7700.00
3261,10997,89,37400.00
3262,10998,77,3500.00
3263,10999,94,9800.00


In [23]:
pensionnaireTwo(ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(year)], ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)])

,id,age,salaire,pension
0,86,65,35390.25,17695.125
1,107,65,39910.50,19955.250
2,110,65,17970.75,8985.375
3,160,65,20286.00,10143.000
4,179,65,16427.25,8213.625
...,...,...,...,...
3979,9949,63,14222.25,7111.125
3980,9952,64,13009.50,6504.750
3981,9957,64,19293.75,9646.875
3982,9986,64,16647.75,8323.875


In [24]:
pensionnaireFive(ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)])

,id,age,salaire,pension
0,86,65,35390.25,17695.125
1,107,65,39910.50,19955.250
2,110,65,17970.75,8985.375
3,160,65,20286.00,10143.000
4,179,65,16427.25,8213.625
...,...,...,...,...
3260,10996,91,7700.00,3850.000
3261,10997,89,37400.00,18700.000
3262,10998,77,3500.00,1750.000
3263,10999,94,9800.00,4900.000


In [25]:
ensemble_data_frames["Simulation_"+str(simulation)]['newRecru_20'+str(year)]

,id,age,salaire,cotisation
0,1,29,7000,490.0
1,2,23,14400,1008.0
2,3,36,3200,160.0
3,4,34,3400,170.0
4,5,34,3300,165.0
...,...,...,...,...
348,349,29,6600,396.0
349,350,31,4400,220.0
350,351,34,4700,235.0
351,352,23,19800,1386.0


In [26]:
calculIndice(1, 62)

,year,TotEmp,TotRet,TotCotis,TotPens,Reserve,NouvRet,NouvRec
0,2023,10000,1000,94662696.00,67929000.0,2.267337e+08,0,0
1,2024,10353,1000,97037100.00,67929000.0,2.558418e+08,0,353
2,2025,10706,1367,97509469.80,118878360.0,2.344729e+08,367,353
3,2026,11059,1771,92046535.20,175107750.0,1.514117e+08,404,353
4,2027,11412,2114,87566597.40,224089620.0,1.488867e+07,343,353
5,2028,11398,2469,82855780.80,274831710.0,-1.770873e+08,355,353
6,2029,11347,2883,77366329.80,331112760.0,-4.308337e+08,414,353
7,2030,11357,3265,76317291.30,394224994.5,-7.487414e+08,382,353
8,2031,11355,3638,71149336.11,448454103.0,-1.126046e+09,373,353
9,2032,11294,3984,66561596.16,498394045.5,-1.557879e+09,346,353


In [27]:
calculIndice(1, 65)

,year,TotEmp,TotRet,TotCotis,TotPens,Reserve,NouvRet,NouvRec
0,2023,10000,1000,9.466270e+07,67929000.0,2.267337e+08,0,0
1,2024,10353,1000,9.703710e+07,67929000.0,2.558418e+08,0,353
2,2025,10706,1000,1.046424e+08,67929000.0,2.925552e+08,0,353
3,2026,11059,1000,1.070516e+08,67929000.0,3.316777e+08,0,353
4,2027,11412,1000,1.094291e+08,67929000.0,3.731778e+08,0,353
5,2028,11398,1367,1.046892e+08,118878360.0,3.589887e+08,367,353
6,2029,11347,1771,9.920703e+07,175107750.0,2.830880e+08,404,353
7,2030,11357,2114,9.979337e+07,226538713.5,1.563426e+08,343,353
8,2031,11355,2469,9.475840e+07,279817908.0,-2.871686e+07,355,353
9,2032,11294,2883,8.888894e+07,338913010.5,-2.787409e+08,414,353
